In [210]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from scipy import stats
import statsmodels.api as sm
from sklearn.linear_model import RidgeCV
from sklearn.metrics import r2_score
cc1 = pd.read_csv("climate_change_1.csv")
cc1_train = cc1[cc1.Year <= 2006]
cc1_test = cc1[cc1.Year > 2006]
cc1.head()


,Year,Month,MEI,CO2,CH4,N2O,CFC-11,CFC-12,TSI,Aerosols,Temp
0,1983,5,2.556,345.96,1638.59,303.677,191.324,350.113,1366.1024,0.0863,0.109
1,1983,6,2.167,345.52,1633.71,303.746,192.057,351.848,1366.1208,0.0794,0.118
2,1983,7,1.741,344.15,1633.22,303.795,192.818,353.725,1366.2850,0.0731,0.137
3,1983,8,1.130,342.25,1631.35,303.839,193.602,355.633,1366.4202,0.0673,0.176
4,1983,9,0.428,340.17,1648.40,303.901,194.392,357.465,1366.2335,0.0619,0.149


# Problem1

## 1.1

In [211]:
def closed_form_1(x,y):
    return np.linalg.inv(x.T @ x) @ (x.T @ y)

In [212]:
x_train = cc1_train[["MEI","CO2","CH4","N2O","CFC-11","CFC-12","TSI","Aerosols"]]
x_train_withc = x_train.copy()
x_train_withc["constant"] = 1
x_train_withc = x_train_withc.values
#x_train = x_train.values
y_train = cc1_train["Temp"].values
theta1 = closed_form_1(x_train_withc,y_train) 
theta1_pd = pd.DataFrame(theta1)
theta1_pd.columns=["coefficient"]
theta1_pd.index=["MEI","CO2","CH4","N2O","CFC-11","CFC-12","TSI","Aerosols","constant"]
theta1_pd

,coefficient
MEI,0.064205
CO2,0.006457
CH4,0.000124
N2O,-0.016528
CFC-11,-0.006630
CFC-12,0.003808
TSI,0.093141
Aerosols,-1.537613
constant,-124.594262


## 1.2

The mathematical formula for the linear model is:
\begin{equation}
y_i = \beta_0 + \beta_1 x_{i1} +\beta_2 x_{i2}+……+\beta_n x_{in}+ \epsilon
\end{equation}
where i is the index of samples

$x_i$ is the expanatory variables

$y_i$ is the dependent variables

$\beta_0$ is y intercept

$\beta_i$ is coefficients for expanatory variables

$\epsilon$ is error term

And $R^2$ of traning set is 0.751 and $R^2$ of test set is 0.184
In this problem, 
\begin{equation}
Temp = 0.064205*MEI+0.006457*CO2+0.000124*CH4+(-0.016528)*N2O+(-0.006630)*CFC-11+0.003808*CFC-12+0.093141*TSI+(-1.537613)*Aerosols-124.594262 + \epsilon
\end{equation}

In [213]:
def r_square(x,y,theta):
    yhat = x @ theta
    ybar = np.mean(y) # 求均值
    #ssreg = np.sum((yhat - ybar)**2)
    sse = np.sum((yhat - y)**2)
    sst = np.sum((y - ybar)**2)
    return 1 - sse / sst

In [214]:
r_square_train = r_square(x_train_withc, y_train, theta1)
print("r square of train set id {}".format(r_square_train))

r square of train set id 0.7508932770523432


In [215]:
x_test = cc1_test[["MEI","CO2","CH4","N2O","CFC-11","CFC-12","TSI","Aerosols"]]
x_test_withc = x_test.copy()
x_test_withc["constant"] = 1
x_test_withc = x_test_withc.values
y_test = cc1_test["Temp"].values

In [216]:
r_square_test = r_square(x_test_withc, y_test, theta1)
print("r square of test set is {}".format(r_square_test))

r square of test set is 0.18377835441261303


## 1.3

use $H_0 : \beta_i = 0$ and $H_1 : \beta_i \not= 0$， here p_values include both sides of distribution
If we choose $\alpha$ = 0.05, then CH4 and N2O is not significant, MEI, CO2

In [217]:
y_train_predict = x_train_withc @ theta1
MSE = (sum((y_train-y_train_predict)**2))/(len(x_train_withc)- np.shape(x_train_withc)[1])
var_b = MSE * (np.linalg.inv(np.dot(x_train_withc.T,x_train_withc)).diagonal())
svar_b = np.sqrt(var_b)
ts_b = theta1/ svar_b
p_values =[2*(1-stats.t.cdf(np.abs(i),(len(x_train_withc)-1))) for i in ts_b]
result = pd.DataFrame()
result["p value"] = p_values
result["t statistic"] = ts_b
result.index=["MEI","CO2","CH4","N2O","CFC-11","CFC-12","TSI","Aerosols","constant"]
result

,p value,t statistic
MEI,0.000000e+00,9.923226
CO2,5.042595e-03,2.826420
CH4,8.101405e-01,0.240469
N2O,5.464020e-02,-1.929726
CFC-11,5.913566e-05,-4.077834
CFC-12,2.085760e-04,3.757293
TSI,1.057310e-09,6.312561
Aerosols,5.109246e-12,-7.210301
constant,1.381915e-09,-6.265174


In [218]:
cc2 = pd.read_csv("climate_change_2.csv")
cc2_train = cc2[cc2.Year <= 2006]
cc2_test = cc2[cc2.Year > 2006]
cc2.head()

,Year,Month,MEI,CO2,CH4,N2O,CFC-11,CFC-12,TSI,Aerosols,NO,Temp
0,1983,5,2.556,345.96,1638.59,303.677,191.324,350.113,1366.1024,0.0863,2.63859,0.109
1,1983,6,2.167,345.52,1633.71,303.746,192.057,351.848,1366.1208,0.0794,2.63371,0.118
2,1983,7,1.741,344.15,1633.22,303.795,192.818,353.725,1366.2850,0.0731,2.63322,0.137
3,1983,8,1.130,342.25,1631.35,303.839,193.602,355.633,1366.4202,0.0673,2.63135,0.176
4,1983,9,0.428,340.17,1648.40,303.901,194.392,357.465,1366.2335,0.0619,2.64840,0.149


## 1.4

The condition is $X^T X$ is invertible. 

climate_change_2.csv has two variables that are highly corelated： NO and CH4 are highly correlated, the $R^2$ of $CH4 = \beta_0 + \beta_1 NO + \epsilon$ is 1. And one of them are rudundant variables.

Then the determinant of $X^T X$ is close to(if one column can be calculated simple linearly（y = a + bx) from another,then equal to 0) 0.In computer calculation, it is easy to have numerical problems. In the process of calculate the invertibility of matrix, the result may bu not exact.

In [219]:
x = cc2[["NO"]]
x["constant"] = 1
x = x.values
y = cc2["CH4"].values
import statsmodels.api as sm
model = sm.OLS(y,x)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.796e+29
Date:                Fri, 27 Dec 2019   Prob (F-statistic):               0.00
Time:                        21:09:14   Log-Likelihood:                 7876.0
No. Observations:                 308   AIC:                        -1.575e+04
Df Residuals:                     306   BIC:                        -1.574e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1          1000.0000   2.36e-12   4.24e+14      0.0

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


# Problem 2

## 2.1

loss function for linear model with L1 regularization:
\begin{equation}
\frac{1}{2m}[\sum_{i=1}^{m}(\hat{y}_i-y_i)^2+\lambda\sum_{j=1}^{n}\left|\theta_j\right|]
\label{eq:with L1 regularization}
\end{equation}
where m is the number of samples, n is the number of explanatory variables


loss function for linear model with L2 regularization:
\begin{equation}
\frac{1}{2m}[\sum_{i=1}^{m}(\hat{y}_i-y_i)^2+\lambda\sum_{j=1}^{n}\theta_j^2]
\label{eq:with L2 regularization}
\end{equation}
where m is the number of samples, n is the number of explanatory variables


## 2.2

Choose Lamda= 0.1

In [220]:
def closed_form_2(x,y,lamda):
    a = np.identity(np.shape(x)[1])
    a[np.shape(x)[1] - 1,np.shape(x)[1] - 1] = 0
    return np.linalg.inv(x.T @ x + lamda * a) @ (x.T @ y)

In [221]:
theta2 = closed_form_2(x_train_withc, y_train, 0.1) 
theta2_pd = pd.DataFrame(theta2)
theta2_pd.columns=["coefficient_with_L2"]
theta2_pd.index=["MEI","CO2","CH4","N2O","CFC-11","CFC-12","TSI","Aerosols","constant"]
theta2_pd

,coefficient_with_L2
MEI,0.058361
CO2,0.007299
CH4,0.000189
N2O,-0.018100
CFC-11,-0.006900
CFC-12,0.003886
TSI,0.087290
Aerosols,-0.997495
constant,-116.507105


## 2.3

The solutions is similar according to theta2_pd_compare. 

Linear model with L2 regularization is robust since it can  reduce values of parameters and prevent overfitting. And L2 is more sensible to the coefficients while higher absolute value. The generalization of the model is better, so it performs better on the test set. The influence of extreme data in train set is less.

In [222]:
theta2_pd_compare = theta2_pd.copy()
theta2_pd_compare["coefficient_without_L2"] = theta1
theta2_pd_compare

,coefficient_with_L2,coefficient_without_L2
MEI,0.058361,0.064205
CO2,0.007299,0.006457
CH4,0.000189,0.000124
N2O,-0.018100,-0.016528
CFC-11,-0.006900,-0.006630
CFC-12,0.003886,0.003808
TSI,0.087290,0.093141
Aerosols,-0.997495,-1.537613
constant,-116.507105,-124.594262


In [223]:
y_train_predict = x_train_withc @ theta2
MSE = (sum((y_train-y_train_predict)**2))/(len(x_train_withc)- np.shape(x_train_withc)[1])
var_b = MSE * (np.linalg.inv(np.dot(x_train_withc.T,x_train_withc)).diagonal())
svar_b = np.sqrt(var_b)
ts_b = theta1/ svar_b
p_values =[2*(1-stats.t.cdf(np.abs(i),(len(x_train_withc)-1))) for i in ts_b]
result = pd.DataFrame()
result["p value"] = p_values
result["t statistic"] = ts_b
result.index=["MEI","CO2","CH4","N2O","CFC-11","CFC-12","TSI","Aerosols","constant"]
result

,p value,t statistic
MEI,0.000000e+00,9.809468
CO2,5.561250e-03,2.794018
CH4,8.122759e-01,0.237713
N2O,5.745452e-02,-1.907604
CFC-11,7.143337e-05,-4.031086
CFC-12,2.455231e-04,3.714220
TSI,1.590465e-09,6.240195
Aerosols,8.515411e-12,-7.127643
constant,2.067904e-09,-6.193351


## 2.4

In [242]:
lamda = [10, 1, 0.1, 0.01, 0.001]
r_square_train_list = []
r_square_test_list = []
for i in lamda:
    theta = closed_form_2(x_train_withc, y_train, i) 
    r_square_train = r_square(x_train_withc, y_train, theta)
    r_square_train_list.append(r_square_train)
    r_square_test = r_square(x_test_withc, y_test, theta)
    r_square_test_list.append(r_square_test)
r_result = pd.DataFrame()
r_result["r_square of train_set"] = r_square_train_list
r_result["r_square of test_set"] = r_square_test_list
r_result.index = lamda
r_result
    
    

,r_square of train_set,r_square of test_set
10.000,0.704115,-0.287933
1.000,0.717278,-0.089198
0.100,0.745082,0.098240
0.010,0.750769,0.173363
0.001,0.750892,0.182719


In [243]:
# 提供一组备选的α值, RidgeCV类选择一个合适的α值.
ridgecv = RidgeCV(alphas=[10, 1, 0.1, 0.01, 0.001], cv=5)
ridgecv.fit(x_train, y_train)
print("when cv = 5, the best lambda is {}".format(ridgecv.alpha_))
ridgecv = RidgeCV(alphas=[10, 1, 0.1, 0.01, 0.001], cv=10)
ridgecv.fit(x_train, y_train)
print("when cv = 10, the best lambda is {}".format(ridgecv.alpha_))

when cv = 5, the best lambda is 0.001
when cv = 10, the best lambda is 0.01


# Problem 3

## 3.1

### First: remove highly correlated features:(remove the feature whose vif is greater than 10), in this step, remove CFC-12, N2O and CF4

Step one:
get the correlation coefficient matrix and calculate $vif_j=\frac{1}{1-R_j^2}$ where ${R_j^2}$ is the r square of the regression between $x_j$ and other features to find the variable that is highly correlated with other variables.

In [253]:
correlation = x_train.corr()
correlation

,MEI,CO2,CH4,N2O,CFC-11,CFC-12,TSI,Aerosols
MEI,1.000000,-0.041147,-0.033419,-0.050820,0.069000,0.008286,-0.154492,0.340238
CO2,-0.041147,1.000000,0.877280,0.976720,0.514060,0.852690,0.177429,-0.356155
CH4,-0.033419,0.877280,1.000000,0.899839,0.779904,0.963616,0.245528,-0.267809
N2O,-0.050820,0.976720,0.899839,1.000000,0.522477,0.867931,0.199757,-0.337055
CFC-11,0.069000,0.514060,0.779904,0.522477,1.000000,0.868985,0.272046,-0.043921
CFC-12,0.008286,0.852690,0.963616,0.867931,0.868985,1.000000,0.255303,-0.225131
TSI,-0.154492,0.177429,0.245528,0.199757,0.272046,0.255303,1.000000,0.052117
Aerosols,0.340238,-0.356155,-0.267809,-0.337055,-0.043921,-0.225131,0.052117,1.000000


In [254]:
# def vif(x):
#     correlation = x.corr()
#     vif = np.zeros(np.shape(x)[1])
#     for i in range(np.shape(x)[1]):
#         for j in range(np.shape(x)[1]):
#             if i!= j :
#                 vif[i] += 1 / (1-correlation.iloc[i,j] ** 2)
#     return vif
def vif(x):
    vif=[]
    for i in range(np.shape(x)[1]-1):
        theta = closed_form_1(x.drop([x.columns[i]],axis=1),x.iloc[:,i]) 
        vif1 = 1/(1-r_square(x.drop([x.columns[i]],axis=1),x.iloc[:,i], theta))
        vif.append(vif1)
    return vif
x_train_1 = x_train.copy()
x_train_1["constant"] =1
vif(x_train_1)

[1.2173189852022297,
 22.98291086231459,
 18.673084776131148,
 55.88969192427592,
 39.18324241424785,
 120.50292550321454,
 1.1795388493824828,
 1.378404273605766]

Step two:
    
    remove CFC-12 since it has biggest vif

In [255]:
x_train_1 = x_train_1.drop(columns = ["CFC-12"])
vif(x_train_1)

[1.2171616885673988,
 22.535502083480637,
 17.56160005749386,
 32.09518739115482,
 4.808665061133019,
 1.1586949495456074,
 1.3770098650757572]

Step there:
    
    remove N2O since it has biggest vif

In [256]:
x_train_1 = x_train_1.drop(columns = ["N2O"])
vif(x_train_1)

[1.2171428002471456,
 6.6215940105910995,
 12.220306785326802,
 3.966497808725914,
 1.1482051230936552,
 1.37010485476195]

Step four:
    
    remove CH4 since it has biggest vif

In [257]:
x_train_1 = x_train_1.drop(columns = ["CH4"])
vif(x_train_1)

[1.2023895542032323,
 1.6196568032921648,
 1.470754387567506,
 1.1479950467948405,
 1.369222909202984]

In [258]:
x_train_1 = x_train_1.drop(columns = ["constant"])
x_train_1.corr()

,MEI,CO2,CFC-11,TSI,Aerosols
MEI,1.000000,-0.041147,0.069000,-0.154492,0.340238
CO2,-0.041147,1.000000,0.514060,0.177429,-0.356155
CFC-11,0.069000,0.514060,1.000000,0.272046,-0.043921
TSI,-0.154492,0.177429,0.272046,1.000000,0.052117
Aerosols,0.340238,-0.356155,-0.043921,0.052117,1.000000


### Second, First: remove  redundant features (not significant),remove CFC-11

Accoding to t statistic and p value, remove CFC-11, and $R^2$ of training set increase significantly which is 0.32569067160230875

In [259]:
x_train_withc_1 =  x_train_1.copy()
x_train_withc_1["constant"] = 1
model = sm.OLS(y_train, x_train_withc_1)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.734
Model:                            OLS   Adj. R-squared:                  0.730
Method:                 Least Squares   F-statistic:                     153.8
Date:                Fri, 27 Dec 2019   Prob (F-statistic):           7.12e-78
Time:                        21:18:21   Log-Likelihood:                 271.03
No. Observations:                 284   AIC:                            -530.1
Df Residuals:                     278   BIC:                            -508.2
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
MEI            0.0626      0.007      9.481      0.0

In [251]:
x_train_1 = x_train_1.drop(columns = ["CFC-11"])
x_train_withc_1 =  x_train_1.copy()
x_train_withc_1["constant"] = 1
model = sm.OLS(y_train,x_train_withc_1)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.734
Model:                            OLS   Adj. R-squared:                  0.730
Method:                 Least Squares   F-statistic:                     192.1
Date:                Fri, 27 Dec 2019   Prob (F-statistic):           7.36e-79
Time:                        21:18:01   Log-Likelihood:                 270.59
No. Observations:                 284   AIC:                            -531.2
Df Residuals:                     279   BIC:                            -512.9
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
MEI            0.0620      0.007      9.438      0.0

0.7336403429131453

In [252]:
x_test_1 = x_test.drop(columns = ["N2O","CH4","CFC-12","CFC-11"])
x_test_withc_1 =  x_test_1.copy()
x_test_withc_1["constant"] = 1
theta1 = closed_form_1(x_train_withc_1, y_train) 
r_square_test = r_square(x_test_withc_1, y_test, theta1)
r_square_test

-0.31032758669797866

## 3.2

### The result of First step is the same as Problem3.1
### Seacond step is remove  redundant features remove CFC-11

Calculate the t statistic and p value, and then remove cfc-11

In [235]:
x_train_2 = x_train.drop(columns = ["N2O","CH4","CFC-12"])
x_train_withc_2 =  x_train_2.copy()
x_train_withc_2["constant"] = 1
#x_train_withc_2 = x_train_withc2.values
theta2 = closed_form_2(x_train_withc_2, y_train, 0.1) 
theta2

array([ 5.65959034e-02,  1.15847400e-02, -3.84173459e-04,  8.07703482e-02,
       -1.01625501e+00, -1.14183923e+02])

In [236]:
y_train_predict_2 = x_train_withc_2 @ theta2
MSE = (sum((y_train-y_train_predict_2)**2))/(len(x_train_withc_2)- np.shape(x_train_withc_2)[1])
var_b = MSE * (np.linalg.inv(np.dot(x_train_withc_2.T,x_train_withc_2)).diagonal())
svar_b = np.sqrt(var_b)
ts_b = theta2 / svar_b
p_values =[2*(1-stats.t.cdf(np.abs(i),(len(x_train_withc_2)-1))) for i in ts_b]
result = pd.DataFrame()
result["p value"] = p_values
result["t statistic"] = ts_b
result.index=["MEI","CO2","CFC-11","TSI","Aerosols","constant"]
result

,p value,t statistic
MEI,1.332268e-15,8.475800
CO2,0.000000e+00,18.394721
CFC-11,2.412117e-01,-1.174426
TSI,1.876498e-07,5.343632
Aerosols,6.250163e-06,-4.604627
constant,6.924271e-08,-5.539814


In [237]:
x_train_2 = x_train_2.drop(columns = ["CFC-11"])
x_train_withc_2 =  x_train_2.copy()
x_train_withc_2["constant"] = 1
#x_train_withc_2 = x_train_withc2.values
theta2 = closed_form_2(x_train_withc_2, y_train, 0.1) 
theta2

array([ 5.57572714e-02,  1.12267974e-02,  7.68937418e-02, -1.03439223e+00,
       -1.08855111e+02])

In [238]:
y_train_predict_2 = x_train_withc_2 @ theta2
MSE = (sum((y_train-y_train_predict_2)**2))/(len(x_train_withc_2)- np.shape(x_train_withc_2)[1])
var_b = MSE * (np.linalg.inv(np.dot(x_train_withc_2.T,x_train_withc_2)).diagonal())
svar_b = np.sqrt(var_b)
ts_b = theta2 / svar_b
p_values =[2*(1-stats.t.cdf(np.abs(i),(len(x_train_withc_2)-1))) for i in ts_b]
result = pd.DataFrame()
result["p value"] = p_values
result["t statistic"] = ts_b
result.index=["MEI","CO2","TSI","Aerosols","constant"]
result

,p value,t statistic
MEI,2.442491e-15,8.386943
CO2,0.000000e+00,20.537701
TSI,3.723907e-07,5.205549
Aerosols,3.816051e-06,-4.714016
constant,1.367613e-07,-5.406462


In [239]:
x_test_2 = x_test.drop(columns = ["N2O","CH4","CFC-12","CFC-11"])
x_test_withc_2 =  x_test_2.copy()
x_test_withc_2["constant"] = 1
lamda = [10, 1, 0.1, 0.01, 0.001]
r_square_train_list = []
r_square_test_list = []
for i in lamda:
    theta = closed_form_2(x_train_withc_2, y_train, i) 
    r_square_train = r_square(x_train_withc_2, y_train, theta)
    r_square_train_list.append(r_square_train)
    r_square_test = r_square(x_test_withc_2, y_test, theta)
    r_square_test_list.append(r_square_test)
r_result = pd.DataFrame()
r_result["r_square of train_set"] = r_square_train_list
r_result["r_square of test_set"] = r_square_test_list
r_result.index = lamda
r_result

,r_square of train_set,r_square of test_set
10.000,0.683703,-1.141471
1.000,0.697505,-0.872407
0.100,0.727503,-0.511775
0.010,0.733511,-0.337445
0.001,0.733639,-0.313134


In [240]:
ridgecv = RidgeCV(alphas=[10, 1, 0.1, 0.01, 0.001], cv=5)
ridgecv.fit(x_train_2, y_train)
print("when cv = 5, the best lambda is {}".format(ridgecv.alpha_))
ridgecv = RidgeCV(alphas=[10, 1, 0.1, 0.01, 0.001], cv=10)
ridgecv.fit(x_train_2, y_train)
print("when cv = 10, the best lambda is {}".format(ridgecv.alpha_))

when cv = 5, the best lambda is 0.01
when cv = 10, the best lambda is 0.001


# Problem 4

The iteration expression is that
\begin{equation}
\theta_j = \theta_j -\frac{1}{m}[\sum_{i=1}^{m}(\hat{y}_i-y_i)x_{ij}]
\label{eq:with L1 regularization}
\end{equation}
where j is the number of parameters and i is the number of samples

In [185]:
def Gradient_Descent(alpha, theta_0, x, y, tol):
    theta = theta_0
    cost_d1 = (1 / len(x)) * (x.T @ (x @ theta_0 - y))
    count = 0
    while not ((np.all(cost_d1) <= tol) | (count > 100000000)):
        count += 1
        theta = theta - alpha * (1 / len(x)) * (x.T @ (x @ theta - y))
        cost_d1 = (1 / len(x)) * (x.T @ (x @ theta - y))
    print('times is {}'.format(count))
    return theta
# def Gradient_Descent1(alpha, theta_0, x, y, tol):
#     theta = theta_0
#     cost_d1 = (1 / 2 * len(x)) * ((x @ theta_0 - y) ** 2)
#     count = 0
#     while not ((np.sum(cost_d1) <= tol) | (count > 10000000)):
#         count += 1
#         theta = theta - alpha * (1 / len(x)) * (x.T @ (x @ theta - y))
#         cost_d1 = (1 / 1 * len(x)) * ((x @ theta - y) ** 2)
#     print('times is {}'.format(count))
#     return theta


In [177]:
x_GD = np.array([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],[2, 5, 7, 8, 6, 10, 13, 17, 18, 19]])
x_GD = np.insert(x_GD, 0, np.ones(10), axis = 0).T
y_GD = np.array([10, 33, 34, 37, 36, 39, 41, 44, 44, 45])
theta_GD = Gradient_Descent(1e-5, np.ones(3), x_GD, y_GD, 1e-5)
theta_GD

times is 10000001


array([21.04187218,  2.06418162,  0.37191704])

In [36]:
import statsmodels.api as sm
model = sm.OLS(y_GD,x_GD)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.682
Model:                            OLS   Adj. R-squared:                  0.591
Method:                 Least Squares   F-statistic:                     7.513
Date:                Tue, 24 Dec 2019   Prob (F-statistic):             0.0181
Time:                        20:12:02   Log-Likelihood:                -31.132
No. Observations:                  10   AIC:                             68.26
Df Residuals:                       7   BIC:                             69.17
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         21.0419      4.445      4.734      0.0

/Users/zhou/.local/lib/python3.6/site-packages/scipy/stats/stats.py:1450: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  "anyway, n=%i" % int(n))
